In [7]:
import json

In [13]:
settings = {
                # Model settings
                'mode': '3D', # Options [3D, 2D]
                'allow_time_extrapolation': False,            # Allow extrapolation of time for fieldset
                'verbose_delete': False,                      # Print extra information when executing delete operations
                
                # Flags
                'mixing_f': False,                             # Turn on/off vertical turbulent mixing
                'biofouling_f': False,                         # Turn on/off biofouling of particles
                'stokes_f': False,                             # Turn on/off Stokes Drift
                'wind_f': False,                               # Turn on/off Windage

                # Ocean model
                'ocean' : {
                    'modelname': 'NEMO0083',
                    'directory': 'data/input_data/MOi/',                 # Directory of ocean model data
                    'filename_style': 'psy4v3r1/psy4v3r1-daily_',          # Filename style of ocean model data
                    'ocean_mesh': 'domain_ORCA0083-N006/coordinates.nc', # File location of ocean mesh
                    'bathymetry_mesh': 'domain_ORCA0083-N006/bathymetry_ORCA12_V3.3.nc', # File location of bathymetry mesh
                    'variables': { #Variable names in the ocean model for velocities, temperature and salinity
                        'U': 'vozocrtx', # Variable name for the U-velocity
                        'V': 'vomecrty', # Variable name for the V-velocity
                        'W': 'vovecrtz', # Variable name for the W-velocity
                        'conservative_temperature': 'votemper', # Variable name for the temperature field
                        'absolute_salinity': 'vosaline' # Variable name for the salinity field
                        }, 
                    'dimensions':{ # The dimensions of the ocean model data, providing f-points for C-grid models
                        'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                        'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                        'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                        'conservative_temperature': {'lon': 'glamf', 'lat': 'gphif','depth': 'depthw', 'time': 'time_counter'},
                        'absolute_salinity': {'lon': 'glamf', 'lat': 'gphif','depth': 'depthw', 'time': 'time_counter'}
                        },
                    'indices': {},
                    'bathymetry_variables': {'bathymetry': 'Bathymetry'},
                    'bathymetry_dimensions': {'lon': 'nav_lon', 'lat': 'nav_lat'},
                    'mixing_variables': {'mixing_kz' : 'votkeavt'},
                    'mixing_dimensions': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}
                },
                
                # Biogeochemistry model
                'bgc' : {
                    'directory': 'data/input_data/MOi/',                # Directory of biogeochemistry model
                    'filename_style': 'biomer4v2r1/biomer4v2r1-weekly_',
                    'bgc_mesh': 'domain_ORCA025-N006/mesh_hgr_PSY4V3_deg.nc', # File location of biogeochemistry model mesh

                    'variables' : {
                        'pp_phyto': 'nppv', # Total Primary Production of Phyto - 'Net primary prodution of biomass expressed as arbon per unit volume in sea water' [mg m-3 day-1] or [milligrams of Carbon per cubic meter per day]
                        'bio_nanophy': 'phy', # Mole concentration of NanoPhytoplankton expressed as carbon in sea water
                        'bio_diatom': 'phy2' # Mole concentration of Diatoms expressed as carbon in sea water
                        },
                    'dimensions' : {
                        'pp_phyto': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                        'bio_nanophy': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                        'bio_diatom': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}
                        },
                    'indices': {},
                    'constants':{
                        'biofilm_density': 1388.,                 # Biofilm density [g m-3]
                        'algae_cell_volume': 2.0E-16,             # Volume of 1 algal cell [m3]
                        'K': 1.0306E-13 / (86400. ** 2.),  # Boltzmann constant [m2 kg d-2 K-1] now [s-2] (=1.3804E-23)
                        'R20': 0.1 / 86400.,         # Respiration rate, now [s-1]
                        'Q10': 2.13,# temperature coefficient respiration [-]
                        'Gamma': 1.728E5 / 86400.,      # Shear frequency [d-1], now [s-1]
                        'carbon_molecular_weight': 12.,              # Atomic weight of Carbon
                        'collision_probability': 1.,              # Collision probability [-]
                        'algae_mortality_rate': 1.,               # TODO: Add description
                        'algae_respiration_f': 1.,                # TODO: Add description
                    }
                },

                # Waves model
                'stokes' : {
                    'directory': 'data/input_data/ERA5/waves/',                  # Directory of Stokes drift model data
                    'filename_style': 'ERA5_global_waves_monthly_',
                    'variables' : {
                        'Stokes_U': 'ust',
                        'Stokes_V': 'vst',
                        'wave_Tp': 'pp1d'
                        },
                    'dimensions': {
                        'lat': 'latitude', # when not using the converted datasets, use 'longitude' otherwise use 'lon'
                        'lon': 'longitude',
                        'time': 'time'
                        },
                    'indices': {}
                },

                # Wind model
                'wind': {
                    'directory': 'data/input_data/ERA5/wind/',                     # Directory of Wind model data
                    'filename_style': 'ERA5_global_wind_monthly_',
                    'variables' : {
                        'Wind_U': 'u10',
                        'Wind_V': 'v10'
                        },
                    'dimensions' : {
                        'lat': 'latitude', # when not using the converted datasets, use 'longitude' otherwise use 'lon'
                        'lon': 'longitude',
                        'time': 'time'
                        },
                    'indices': {}
                },
                # Unbeaching data
                'unbeaching': {
                    'directory': 'data/output_data/masks/',
                    'filename': 'land_current_NEMO0083.nc',
                    'variables': {'unbeach_U': 'land_current_u',
                                  'unbeach_V': 'land_current_v'},
                    'dimensions': {'lat': 'lat',
                                   'lon': 'lon'}
                },



                'simulation': {
                    'start_date': None,
                    'runtime': None,
                    'dt_write': None,
                    'dt_timestep': None
                },

                'release_maps': {
                    'coastal':'data/release/generated_files/coastal_population_MPW_NEMO0083.csv',
                    'rivers':'data/release/generated_files/river_emissions_NEMO0083.csv',
                    'fisheries':'data/release/generated_files/agg_data_fisheries_info.csv',
                    'global_concentrations':None #Not implemented yet
                }

                }

In [14]:
with open("default_settings.json", "w") as write_file:
    json.dump(settings, write_file, indent=4)

In [15]:
with open("local_settings.json", "w") as write_file:
    json.dump(settings, write_file, indent=4)

In [16]:
settings = {
                    # Model settings
                    'mode': '3D', # Options [3D, 2D]
                    'allow_time_extrapolation': False,            # Allow extrapolation of time for fieldset
                    'verbose_delete': False,                      # Print extra information when executing delete operations
                    
                    # Flags
                    'mixing_f': False,                             # Turn on/off vertical turbulent mixing
                    'biofouling_f': False,                         # Turn on/off biofouling of particles
                    'stokes_f': False,                             # Turn on/off Stokes Drift
                    'wind_f': False,                               # Turn on/off Windage

                    # Ocean model
                    'ocean' : {
                        'modelname': 'NEMO0083',
                        'directory': '/storage/shared/oceanparcels/input_data/MOi/',                 # Directory of ocean model data
                        'filename_style': 'psy4v3r1/psy4v3r1-daily_',          # Filename style of ocean model data
                        'ocean_mesh': 'domain_ORCA0083-N006/Old/coordinates.nc', # File location of ocean mesh
                        'bathymetry_mesh': 'domain_ORCA0083-N006/Old/bathymetry_ORCA12_V3.3.nc', # File location of bathymetry mesh
                        'variables': { #Variable names in the ocean model for velocities, temperature and salinity
                            'U': 'vozocrtx', # Variable name for the U-velocity
                            'V': 'vomecrty', # Variable name for the V-velocity
                            'W': 'vovecrtz', # Variable name for the W-velocity
                            'conservative_temperature': 'votemper', # Variable name for the temperature field
                            'absolute_salinity': 'vosaline' # Variable name for the salinity field
                            }, 
                        'dimensions':{ # The dimensions of the ocean model data, providing f-points for C-grid models
                            'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                            'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                            'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                            'conservative_temperature': {'lon': 'glamf', 'lat': 'gphif','depth': 'depthw', 'time': 'time_counter'},
                            'absolute_salinity': {'lon': 'glamf', 'lat': 'gphif','depth': 'depthw', 'time': 'time_counter'}
                            },
                        'indices': {},
                        'bathymetry_variables': {'bathymetry': 'Bathymetry'},
                        'bathymetry_dimensions': {'lon': 'nav_lon', 'lat': 'nav_lat'},
                        'mixing_variables': {'mixing_kz' : 'votkeavt'},
                        'mixing_dimensions': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}
                    },
                    
                    # Biogeochemistry model
                    'bgc' : {
                        'directory': '/storage/shared/oceanparcels/input_data/MOi/',                # Directory of biogeochemistry model
                        'filename_style': 'biomer4v2r1/biomer4v2r1-weekly_',
                        'bgc_mesh': 'domain_ORCA025-N006/mesh_hgr_PSY4V3_deg.nc', # File location of biogeochemistry model mesh

                        'variables' : {
                             'pp_phyto': 'nppv', # Total Primary Production of Phyto - 'Net primary prodution of biomass expressed as arbon per unit volume in sea water' [mg m-3 day-1] or [milligrams of Carbon per cubic meter per day]
                            'bio_nanophy': 'phy', # Mole concentration of NanoPhytoplankton expressed as carbon in sea water
                            'bio_diatom': 'phy2' # Mole concentration of Diatoms expressed as carbon in sea water
                            },
                        'dimensions' : {
                            'pp_phyto': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                            'bio_nanophy': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
                            'bio_diatom': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}
                            },
                        'indices': {},
                        'constants':{
                            'biofilm_density': 1388.,                 # Biofilm density [g m-3]
                            'algae_cell_volume': 2.0E-16,             # Volume of 1 algal cell [m3]
                            'K': 1.0306E-13 / (86400. ** 2.),  # Boltzmann constant [m2 kg d-2 K-1] now [s-2] (=1.3804E-23)
                            'R20': 0.1 / 86400.,         # Respiration rate, now [s-1]
                            'Q10': 2.13,# temperature coefficient respiration [-]
                            'Gamma': 1.728E5 / 86400.,      # Shear frequency [d-1], now [s-1]
                            'carbon_molecular_weight': 12.,              # Atomic weight of Carbon
                            'collision_probability': 1.,              # Collision probability [-]
                            'algae_mortality_rate': 1.,               # TODO: Add description
                            'algae_respiration_f': 1.,                # TODO: Add description
                        }
                    },

                    # Waves model
                    'stokes' : {
                        'directory': '/storage/shared/oceanparcels/output_data/data_Michael/ERA5/waves/',                  # Directory of Stokes drift model data
                        'filename_style': 'ERA5_global_waves_monthly_',
                        'variables' : {
                            'Stokes_U': 'ust',
                            'Stokes_V': 'vst',
                            'wave_Tp': 'pp1d'
                            },
                        'dimensions': {
                            'lat': 'latitude', # when not using the converted datasets, use 'longitude' otherwise use 'lon'
                            'lon': 'longitude',
                            'time': 'time'
                            },
                        'indices': {}
                    },

                    # Wind model
                    'wind': {
                        'directory': '/storage/shared/oceanparcels/output_data/data_Michael/ERA5/wind/',                     # Directory of Wind model data
                        'filename_style': 'ERA5_global_wind_monthly_',
                        'variables' : {
                            'Wind_U': 'u10',
                            'Wind_V': 'v10'
                            },
                        'dimensions' : {
                            'lat': 'latitude', # when not using the converted datasets, use 'longitude' otherwise use 'lon'
                            'lon': 'longitude',
                            'time': 'time'
                            },
                        'indices': {}
                    },
                    # Unbeaching data
                    'unbeaching': {
                        'directory': 'data/output_data/',
                        'filename': 'land_current_NEMO0083.nc',
                        'variables': {'unbeach_U': 'land_current_u',
                                    'unbeach_V': 'land_current_v'},
                        'dimensions': {'lat': 'lat',
                                    'lon': 'lon'}
                    },

                    'simulation': {
                        'start_date': None,
                        'runtime': None,
                        'dt_write': None,
                        'dt_timestep': None
                    },

                    'release_maps': {
                        'coastal':'data/output_data/coastal_population_MPW_NEMO0083.csv',
                        'rivers':'data/output_data/river_emissions_NEMO0083.csv',
                        'fisheries':'data/output_data/agg_data_fisheries_info.csv',
                        'global_concentrations':None #Not implemented yet
                    }

                    }


In [17]:
with open("lorenz_settings.json", "w") as write_file:
    json.dump(settings, write_file, indent=4)